###  5.4.1 持久化代码实现
使用tf.train.Saver()类来保存和还原一个神经网络模型

In [1]:
import tensorflow as tf

In [2]:
# 声明两个变量并计算它们的和
v1 = tf.Variable(tf.constant(1.0, shape=[1], name="v1"))
v2 = tf.Variable(tf.constant(1.0, shape=[1], name="v2"))
result = v1 + v2

In [7]:
init_op = tf.global_variables_initializer()
# 声明tf.train.Saver()用于保存模型
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init_op)
    # 保存模型
    saver.save(sess, "Saved_model/model.ckpt")

虽然上述程序只指定了一个文件路径，但是保存了3个文件，因为tensorflow会将计算图的结构和图上参数取值分开保存。
- model.ckpt.meta 保存计算图的结构
- model.ckpt 保存程序中每个变量的取值
- checkpoint 保存一个目录下所有模型文件列表

In [9]:
# 直接加载模型

with tf.Session() as sess:
    saver.restore(sess, "Saved_model/model.ckpt")
    print(sess.run(result))

INFO:tensorflow:Restoring parameters from Saved_model/model.ckpt
[ 2.]


In [12]:
# 直接加载持久化的图
saver = tf.train.import_meta_graph("Saved_model/model.ckpt.meta")

In [13]:
with tf.Session() as sess:
    saver.restore(sess, "Saved_model/model.ckpt")
    print(sess.run(tf.get_default_graph().get_tensor_by_name("add:0")))

INFO:tensorflow:Restoring parameters from Saved_model/model.ckpt
[ 2.]


 tf.train.Saver()支持在保存或加载时给变量重命名。

In [14]:
v1 = tf.Variable(tf.constant(1.0, shape=[1], name="other-v1"))
v2 = tf.Variable(tf.constant(1.0, shape=[1], name="other-v2"))

In [16]:
saver = tf.train.Saver({"v1":v1, "v2":v2})

保存滑动平均模型

In [29]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [33]:
import tensorflow as tf

# 清理图中保存的变量，重置图
tf.reset_default_graph()

v = tf.Variable(0, dtype = tf.float32, name="v")
# 在没有申明滑动平均模型时只有一个变量，所以下面的语句只会输出“v:0”
for variables in tf.global_variables():
    print(variables.name)

v:0


In [34]:
ema = tf.train.ExponentialMovingAverage(0.99)
maintain_average_op = ema.apply(tf.global_variables())
# 在申明滑动平均模型之后会自动生成一个影子变量

for variables in tf.global_variables():
    print(variables.name)

v:0
v/ExponentialMovingAverage:0


In [35]:
saver = tf.train.Saver()
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    sess.run(tf.assign(v,10))
    sess.run(maintain_average_op)
    saver.save(sess, "Saved_model/model.ckpt")
    print(sess.run([v, ema.average(v)]))

[10.0, 0.099999905]


使用variables_to_restore()函数生成tf.train.Saver()类所需要的变量重命名字典。

In [36]:
import tensorflow as tf

# 清理图中保存的变量，重置图
tf.reset_default_graph()

v = tf.Variable(0, dtype = tf.float32, name="v")
ema = tf.train.ExponentialMovingAverage(0.99)

print(ema.variables_to_restore())

saver = tf.train.Saver(ema.variables_to_restore())
with tf.Session() as sess:
    saver.restore(sess, "Saved_model/model.ckpt")
    print(sess.run(v))

{'v/ExponentialMovingAverage': <tf.Variable 'v:0' shape=() dtype=float32_ref>}
INFO:tensorflow:Restoring parameters from Saved_model/model.ckpt
0.0999999
